In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import gc
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import IsolationForest

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
def plot_outliers(x,f,t):
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.set_title(t)
    ax.scatter(x=x[f[0]], y=x[f[1]],c=x['outlier'])
    plt.show()

In [ ]:
def predict(x, contamination= 0.01):
    sklIF = IsolationForest(behaviour= "new",
                            contamination = contamination,
                            random_state = np.random.RandomState(42))
    return sklIF.fit_predict(x)


In [ ]:
building_metadata = pd.read_csv("../input/ashrae-energy-prediction/building_metadata.csv")
building_metadata = reduce_mem_usage(building_metadata)

train = pd.read_csv("../input/ashrae-energy-prediction/train.csv",parse_dates=['timestamp'])
train = reduce_mem_usage(train)

weather_train = pd.read_csv("../input/ashrae-energy-prediction/weather_train.csv",parse_dates=['timestamp'])
weather_train = reduce_mem_usage(weather_train)

### Data Cleaning

In [ ]:
train = train.query('not (building_id <= 104 & meter == 0 & timestamp <= "2016-05-20")')

### Feature Engineering

In [ ]:
building_metadata["year_built_fe"] = building_metadata["year_built"].fillna(1960) - 1900
building_metadata["year_built_fe"] = building_metadata["year_built_fe"] // 10

In [ ]:
le = LabelEncoder()
building_metadata["primary_use"] = le.fit_transform(building_metadata["primary_use"])

In [ ]:
t = train.merge(building_metadata, on='building_id', how='left')
t = t.merge(weather_train, on=['site_id', 'timestamp'], how='left')

In [ ]:
t["date"]    = t["timestamp"].dt.floor('D')
t['week']    = t['timestamp'].dt.week
t['weekday'] = t['timestamp'].dt.weekday
t["hour"]    = t["timestamp"].dt.hour
t["log_meter_reading"] = np.log1p(t['meter_reading'])

t['air_temperature'] = t['air_temperature'].interpolate(method='linear')
t['dew_temperature'] = t['dew_temperature'].interpolate(method='linear')

### dew_temperature vs air_temperature outliers

In [ ]:
cont_features = ['dew_temperature',
                 'air_temperature' ]

In [ ]:
for i in range(16):
    x = t[t.site_id == i].copy()
    x['outlier'] = predict(x[cont_features])
    plot_outliers(x,cont_features,f'site_id = {i}')

### square_feet vs log_meter_reading outliers

In [ ]:
cont_features = ['square_feet', 'log_meter_reading' ]

In [ ]:
for i in range(4):
    x = t[t.meter == i].copy() 
    x['outlier'] = predict(x[cont_features], contamination=0.05)
    plot_outliers(x,cont_features,f'meter = {i}') 